In [1]:
VK_CONFIG = {
    'domain': "https://api.vk.com/method",
    'access_token': "ac3ff17fb5d83ab12a342e177697a25ea9f353049d61449e69e180b7b308563877631c3eeb44a1d2d6700",
    'version': "5.103",
}


PLOTLY_CONFIG = {
    'username': 'Имя пользователя Plot.ly',
    'api_key': 'Ключ доступа Plot.ly',
}


In [2]:

def get_friends(user_id, fields=''):
    """ Вернуть данных о друзьях пользователя

    :param user_id: идентификатор пользователя, список друзей которого нужно получить
    :param fields: список полей, которые нужно получить для каждого пользователя
    """
    assert isinstance(user_id, int), "user_id must be positive integer"
    assert isinstance(fields, str), "fields must be string"
    assert user_id > 0, "user_id must be positive integer"

    query = (f"{VK_CONFIG['domain']}/" +
            "friends.get?" +
            f"access_token={VK_CONFIG['access_token']}&" +
            f"user_id={user_id}&" +
            f"fields={fields}&" +
            f"v={VK_CONFIG['version']}")
    response = requests.get(query)

    return response.json()


In [3]:
from igraph import Graph, plot
import numpy as np

from api import get_friends

def get_network(users_ids, as_edgelist=True):
    vertices = set()
    graph = {}

    for user_id in users_ids:
        vertices.add(user_id)

        friends = get_friends(user_id)

        if not 'error' in friends:
            for user_friend in friends['response']['items']:
                vertices.add(user_friend)

    vertices = [vert for vert in vertices]

    if as_edgelist:
        edges = []

        for user_id in vertices:
            friends = get_friends(user_id)

            if not 'error' in friends:
                for user_friend in friends['response']['items']:
                    if user_friend in vertices:
                        edges.append((user_id, user_friend))
        
        return edges

    else:
        pass
    return graph

def plot_graph(graph):
    get_network([364629895, 313489913, 307593556, 342841693, 511527390])


In [4]:
edges = get_network([556991512, 244018833])

In [5]:
vertices = set()

In [6]:
for i in edges:
    vertices.add(i[0])

In [7]:
vertices = [vert for vert in vertices]

In [8]:
g = Graph(vertex_attrs={"label":vertices},
    edges=edges, directed=False)

In [9]:
g.simplify(multiple=True, loops=True)

In [ ]:
communities = g.community_edge_betweenness(directed=False)
clusters = communities.as_clustering()

In [ ]:
pal = igraph.drawing.colors.ClusterColoringPalette(len(clusters))
g.vs['color'] = pal.get_many(clusters.membership)

In [ ]:
bplot(g)

In [69]:
from normilize_text import normilize_texts
from gensim.models.ldamulticore import LdaModel
from gensim.corpora.dictionary import Dictionary

In [73]:
common_texts = normilize_texts("itmoru")
common_texts = [text.split() for text in common_texts]

In [75]:
common_dictionary = Dictionary([text.split()])
common_corpus = [common_dictionary.doc2bow(text) for text in common_texts]
lda = LdaModel(common_corpus, num_topics=10)

In [77]:
other_texts = [common_texts[0]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]
unseen_doc = other_corpus[0]

In [78]:
vector = lda[unseen_doc]

In [79]:
vector

[(0, 0.90042573), (3, 0.089816466)]